# Siametric Assignment

use R in jupyter

## Data

In [1]:
library(dplyr)
# Set working directory
setwd("/Users/vitawat/desktop/Rdata/siametrics_asg2018/")
# Read files
performance  <- read.csv("performance.csv", head = T)
performance_c <- read.csv("performance_c.csv", head = T)
absence <- read.csv("absence.csv", head = T)
department <- read.csv("department.csv", head = T)
# Change column name
colnames(department)[1] <- "person_id"
str(performance)
str(performance_c)
str(absence)
str(department)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



'data.frame':	20760 obs. of  4 variables:
 $ person_id: int  2 2 2 2 2 30 30 30 30 30 ...
 $ year     : int  2013 2014 2015 2016 2017 2013 2014 2015 2016 2017 ...
 $ perf_a   : num  -0.291 NA NA NA NA ...
 $ perf_b   : num  -0.114 NA NA NA NA ...
'data.frame':	13376 obs. of  3 variables:
 $ person_id: int  18946 47288 120700 63124 128302 42206 64730 116407 83868 59511 ...
 $ year     : int  2013 2013 2014 2017 2017 2017 2016 2015 2014 2016 ...
 $ perf_c   : num  -4.56 -4.55 -4.22 -4.18 -4 ...
'data.frame':	21218 obs. of  4 variables:
 $ person_id   : int  73734 125657 433 123918 94031 86808 49049 37562 58713 30657 ...
 $ year        : int  2014 2014 2014 2014 2014 2014 2014 2014 2014 2014 ...
 $ status      : Factor w/ 4 levels "","Active","Inactive",..: 2 1 2 2 2 2 2 2 2 1 ...
 $ avg_absences: num  9.03 9.03 9.03 9.03 9.03 ...
'data.frame':	14579 obs. of  5 variables:
 $ person_id: int  2 2 2 4 4 4 4 35 35 35 ...
 $ year     : int  2012 2013 2014 2012 2013 2014 2015 2012 2013 2014 ...

## Merge Data
Merge all files together by person_id, id_s_fr, and year.

In [2]:
# Make key index
M1 <- bind_rows(performance[,1:2],performance_c[,1:2], absence[,1:2], department[,1:2])
# Take duplicate person_id and year off 
Mtotal <- M1[!duplicated(M1[,1:2]),]       # Key index
# Before merge -> add varaible to indicate the source data
performance$performance <- 1
performance_c$performance_c <- 1
absence$absence <- 1
department$department <- 1
# Merge files - left_join by "person_id" and "year"
Mtotal  <- Mtotal %>%
  left_join(performance, by = c("person_id", "year")) %>%  
  left_join(performance_c, by = c("person_id", "year")) %>%
  left_join(absence, by = c("person_id", "year")) %>%
  left_join(department, by = c("person_id", "year")) 
str(Mtotal)
nrow(Mtotal)
nrow(subset(Mtotal,performance == 1))
nrow(subset(Mtotal,performance_c == 1))
nrow(subset(Mtotal,absence == 1))
nrow(subset(Mtotal,department == 1))
# write file
write.csv(Mtotal, file="panal.csv", row.names = T)

'data.frame':	29594 obs. of  14 variables:
 $ person_id    : int  2 2 2 2 2 30 30 30 30 30 ...
 $ year         : int  2013 2014 2015 2016 2017 2013 2014 2015 2016 2017 ...
 $ perf_a       : num  -0.291 NA NA NA NA ...
 $ perf_b       : num  -0.114 NA NA NA NA ...
 $ performance  : num  1 1 1 1 1 1 1 1 1 1 ...
 $ perf_c       : num  -0.193 NA NA NA NA ...
 $ performance_c: num  1 NA NA NA NA 1 NA NA NA NA ...
 $ status       : Factor w/ 4 levels "","Active","Inactive",..: 3 3 NA NA NA 1 1 NA NA NA ...
 $ avg_absences : num  8.65 8.86 NA NA NA ...
 $ absence      : num  1 1 NA NA NA 1 1 NA NA NA ...
 $ id_s_fr      : num  NA NA NA NA NA NA NA NA NA NA ...
 $ age          : num  49 50 NA NA NA NA NA NA NA NA ...
 $ female       : num  1 1 NA NA NA NA NA NA NA NA ...
 $ department   : num  1 1 NA NA NA NA NA NA NA NA ...


[1] 29594

[1] 20760

[1] 13376

[1] 21218

[1] 14579

## Q1 : Fill every area in the below Venn diagram with the number of unique people.
1. People in each data file
check person_id appear in each data file

Uperf <- performance[!duplicated(performance[,1]),] # take duplicate id off
Uperfc <- performance_c[!duplicated(performance_c[,1]),] # take duplicate id off
Uabsence  <- absence[!duplicated(absence[,1]),] # take duplicate id off
Udepartment <-department[!duplicated(department[,1]),] # take duplicate id off
nrow(Uperf)
nrow(Uperfc)
nrow(Uabsence)
nrow(Udepartment)

2. Total people (universe)

In [4]:
# Prepare data
M2 <- Mtotal[!duplicated(Mtotal[,1]),]
M3 <- as.data.frame(M2$person_id)
M3$performance <- M2$performance
M3$performance_c <- M2$performance_c
M3$absence <- M2$absence
M3$department <- M2$department
nrow(M3)

[1] 6729

3. Unique people in each area

In [5]:
# Find A <- Only appear in performance
M3$hcheck <- 0
M3$check[M3$performance == 1 & is.na(M3$performance_c) & is.na(M3$absence)  & is.na(M3$department)] <- 1
sum(na.omit(M3$check))

[1] 1064

In [6]:
# Find B <- Only appear in performance and performance_c
M3$check <- 0
M3$check[M3$performance == 1 & M3$performance_c == 1 & is.na(M3$absence)  & is.na(M3$department)] <- 1
sum(na.omit(M3$check))

[1] 58

In [7]:
# Find C <- Only appear in performance_c
M3$check <- 0
M3$check[is.na(M3$performance) & M3$performance_c == 1 & is.na(M3$absence)  & is.na(M3$department)] <- 1
sum(na.omit(M3$check))

[1] 0

In [8]:
# Find D <- Only appear in performance_c and absence
M3$check <- 0
M3$check[is.na(M3$performance) & M3$performance_c == 1 & M3$absence == 1  & is.na(M3$department)] <- 1
sum(na.omit(M3$check))

[1] 0

In [9]:
# Find E <- Only appear in performance_c and absence and department
M3$check <- 0
M3$check[is.na(M3$performance) & M3$performance_c == 1 & M3$absence == 1  & M3$department == 1] <- 1
sum(na.omit(M3$check))

[1] 0

In [10]:
# Find F <- Only appear in absence and department***** not in the answer
M3$check <- 0
M3$check[is.na(M3$performance) & is.na(M3$performance_c) & M3$absence == 1  & M3$department == 1] <- 1
sum(na.omit(M3$check))

[1] 810

In [11]:
# Find G <- Only appear in performance and performance_c and department
M3$check <- 0
M3$check[M3$performance == 1 & M3$performance_c == 1 & is.na(M3$absence)  & M3$department == 1] <- 1
sum(na.omit(M3$check))

[1] 1

In [12]:
# Find H <- appear in performance and performance_c and absence and department
M3$check <- 0
M3$check[M3$performance == 1 & M3$performance_c == 1 & M3$absence == 1 & M3$department == 1] <- 1
sum(na.omit(M3$check))

[1] 2401

In [13]:
# Find I <- Only appear in performance and performance_c and absence
M3$check <- 0
M3$check[M3$performance == 1 & M3$performance_c == 1 & M3$absence == 1 & is.na(M3$department)] <- 1
sum(na.omit(M3$check))

[1] 62

In [14]:
# Find J <- Only appear in performance and absence
M3$check <- 0
M3$check[M3$performance == 1 & is.na(M3$performance_c) & M3$absence == 1 & is.na(M3$department)] <- 1
sum(na.omit(M3$check))

[1] 30

In [15]:
# Find K <- Only appear in absence
M3$check <- 0
M3$check[is.na(M3$performance) & is.na(M3$performance_c) & M3$absence == 1 & is.na(M3$department)] <- 1
sum(na.omit(M3$check))

[1] 1736

In [16]:
# Find L <- Only appear in performance and absence and department
M3$check <- 0
M3$check[M3$performance == 1 & is.na(M3$performance_c) & M3$absence == 1 & M3$department== 1] <- 1
sum(na.omit(M3$check))

[1] 535

In [17]:
# Find M <- Only appear in performance and department
M3$check <- 0
M3$check[M3$performance == 1 & is.na(M3$performance_c) & is.na(M3$absence) & M3$department== 1] <- 1
sum(na.omit(M3$check))

[1] 1

In [18]:
# Find N <- Only appear in department
M3$check <- 0
M3$check[is.na(M3$performance) & is.na(M3$performance_c) & is.na(M3$absence) & M3$department== 1] <- 1
sum(na.omit(M3$check))

[1] 31

In [19]:
# Find O <- Only appear in performance_c and department
M3$check <- 0
M3$check[is.na(M3$performance) & M3$performance_c == 1 & is.na(M3$absence) & M3$department== 1] <- 1
sum(na.omit(M3$check))

[1] 0

## Q2 : Are there any features we should worry about? Check for missingness and other anomalies. Report the top three issues with the data I should worry about.

1. perf_a, perf_b,and perF_c : There are a lot of missing data. It may be get a problem with the total score of each person.

2. All performance score is both positive and negative. Score probabilities derived from standardized.  

3. The data in each row is the data of each person in each period. Be careful when use, must to understand the data.

In [20]:
summary(Mtotal)

   person_id           year          perf_a           perf_b      
 Min.   :     2   Min.   :2012   Min.   :-6.539   Min.   :-3.748  
 1st Qu.: 48098   1st Qu.:2013   1st Qu.:-0.563   1st Qu.:-0.624  
 Median : 79425   Median :2015   Median : 0.046   Median :-0.026  
 Mean   : 79067   Mean   :2352   Mean   : 0.004   Mean   : 0.000  
 3rd Qu.:113976   3rd Qu.:2016   3rd Qu.: 0.612   3rd Qu.: 0.596  
 Max.   :134862   Max.   :9999   Max.   : 5.724   Max.   : 4.013  
                                 NA's   :16517    NA's   :26441   
  performance       perf_c       performance_c         status     
 Min.   :1      Min.   :-4.561   Min.   :1                : 3703  
 1st Qu.:1      1st Qu.:-0.520   1st Qu.:1       Active   :17095  
 Median :1      Median : 0.039   Median :1       Inactive :  419  
 Mean   :1      Mean   :-0.014   Mean   :1       Withdrawn:    1  
 3rd Qu.:1      3rd Qu.: 0.553   3rd Qu.:1       NA's     : 8376  
 Max.   :1      Max.   : 5.724   Max.   :1                    

## Q3: Create a new variable for a person’s average lifetime performance score (pick any of the three scores we have). Call this new variable, lperf. What is the average of lperf among female employees? 

1. Select female = 1 from merged file 

2. Select perf_c to create mew variable <- lperf

3. Find person’s average lifetime performance score = lperf of each people

4. Find mean of lperf = the average of lperf among female employees = 0.009869174

In [21]:
# Find person_id that female == 1 from Mtotal
M1 <- Mtotal
M1 <- M1[!(M1$female == 0 | is.na(M1$female)),]   # select only female
# Find lifetime performance score for each person (use perf_c)
M2 <- aggregate(perf_c ~ person_id, M1, mean)
# Find the average of lperf among female employees 
mean(M2$perf_c)

[1] 0.009869174

## Q4: Transform the data from the current long format (each row is for each person-year record so there’ll be many rows per one person) into the wide format (each row is for each person). Save this as wide.csv.

1. Split data file by year : 2012-2017 and 9999***(keep to check with previous file)

2. Merge year file by column use person_id as key index

3. Save to wide.csv  


In [22]:
# Check year
table(Mtotal$year)
# Cut performance, performan_c, absence, department
Stotal <- Mtotal
Stotal$performance <- NULL
Stotal$performance_c <- NULL
Stotal$absence <- NULL
Stotal$department <- NULL
Stotal$lperf <- NULL
# Create sub file for each year
y2012 <- Stotal[0,]
y2013 <- Stotal[0,]
y2014 <- Stotal[0,]
y2015 <- Stotal[0,]
y2016 <- Stotal[0,]
y2017 <- Stotal[0,]
y9999 <- Stotal[0,]
# Start to split file
i  <- 1
for (i in 1:nrow(Stotal)) {
  if(Stotal$year[i] == 2012){
    y2012 <- rbind(y2012,Stotal[i,])
  }
}
i  <- 1
for (i in 1:nrow(Stotal)) {
  if(Stotal$year[i] == 2013){
    y2013 <- rbind(y2013,Stotal[i,])
  }
}
i  <- 1
for (i in 1:nrow(Stotal)) {
  if(Stotal$year[i] == 2014){
    y2014 <- rbind(y2014,Stotal[i,])
  }
}
i  <- 1
for (i in 1:nrow(Stotal)) {
  if(Stotal$year[i] == 2015){
    y2015 <- rbind(y2015,Stotal[i,])
  }
}
i  <- 1
for (i in 1:nrow(Stotal)) {
  if(Stotal$year[i] == 2016){
    y2016 <- rbind(y2016,Stotal[i,])
  }
}
i  <- 1
for (i in 1:nrow(Stotal)) {
  if(Stotal$year[i] == 2017){
    y2017 <- rbind(y2017,Stotal[i,])
  }
}
i  <- 1
for (i in 1:nrow(Stotal)) {
  if(Stotal$year[i] == 9999){
    y9999 <- rbind(y9999,Stotal[i,])
  }
}
# Maker person_id as key index
Wtotal <- as.data.frame(M3[,1])
colnames(Wtotal)[1] <- "person_id"
# Merge
Wtotal  <- Wtotal %>%
  left_join(y2012, by = "person_id") %>%  
  left_join(y2013, by = "person_id") %>%  
  left_join(y2014, by = "person_id") %>%  
  left_join(y2015, by = "person_id") %>%  
  left_join(y2016, by = "person_id") %>%  
  left_join(y2017, by = "person_id") %>%  
  left_join(y9999, by = "person_id")  
str(Wtotal)
# write file
write.csv(Wtotal, file="wide.csv", row.names = T)


2012 2013 2014 2015 2016 2017 9999 
4056 5200 5256 4850 4828 4152 1252 

'data.frame':	6729 obs. of  64 variables:
 $ person_id         : int  2 30 416 1938 1966 2712 2951 3834 4257 4352 ...
 $ year.x            : int  2012 2012 2012 NA 2012 2012 2012 2012 2012 NA ...
 $ perf_a.x          : num  NA NA NA NA NA NA NA NA NA NA ...
 $ perf_b.x          : num  NA NA NA NA NA NA NA NA NA NA ...
 $ perf_c.x          : num  NA NA NA NA NA NA NA NA NA NA ...
 $ status.x          : Factor w/ 4 levels "","Active","Inactive",..: 2 1 2 NA 2 2 2 2 2 NA ...
 $ avg_absences.x    : num  7.95 8.06 8.65 NA 7.96 ...
 $ id_s_fr.x         : num  3e+09 NA 3e+09 NA 3e+09 ...
 $ age.x             : num  48 NA 73 NA 52 69 68 47 69 NA ...
 $ female.x          : num  1 NA 1 NA 0 1 0 0 1 NA ...
 $ year.y            : int  2013 2013 2013 2013 2013 2013 2013 2013 2013 2013 ...
 $ perf_a.y          : num  -0.291 -0.154 -0.04 NA NA ...
 $ perf_b.y          : num  -0.114 NA 0.206 -0.463 NA ...
 $ perf_c.y          : num  -0.1933 -0.15396 0.00636 -0.28645 NA ...
 $ status.y          : Facto

## Q5 (BONUS): Go back to the long format file, panel.csv. Calculate the rolling average performance score (pick any of the three) for all previous years up until the year for that record. For example, if that row is year =2014, calculate this average using performance score from all years <=2014. If there are missing scores in some year(s) in between, use the latest available rolling average. Call this variable, rperf. What’s the average rperf among male employees in year 2015?
Rolling score year X is average performance score from started year to year X. For this data set, Rolling score year 2015 is average performance score from 2012 to 2015.
1. Use perf_c to calculate rperf 
2. Use wide.csv
3. Select people who is male
4. Find rolling score 2015 (anerage perf_c from 2012-2015)
5. Average of rolling score is the average rperf among male employees in year 2015 = -0.2539987

In [23]:
# Select male from Wtota,(wide.csv) (1 = female, 0 = male)
M4 <- Wtotal[!(Wtotal$female.x == 1 | is.na(Wtotal$female.x)),]   # select only male
M5 <- as.data.frame(M4[,1])
M5$perf_c_2012 <- M4$perf_c.x
M5$perf_c_2013 <- M4$perf_c.y
M5$perf_c_2014 <- M4$perf_c.x.x
M5$perf_c_2015 <- M4$perf_c.y.y
# Calculate rperf for 2015
M5$rperf <- rowMeans(M5[-1], na.rm = TRUE)
M5$rperf[is.nan(M5$rperf)] <- NA
mean(M5$rperf, na.rm = TRUE)

[1] -0.2539987